In [2]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os 
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import itertools

from multiprocessing import Pool


C:\Users\jhroe\AppData\Local\Temp\ipykernel_16260\1407334459.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:


# date and cell cutoff
cutoff = 25

# specify paths

data_path = f'{os.getcwd()}/Data/'
file_path = f"{data_path}/distances/cell_distances_25px_20240506_allpatients.csv"
file_path2 = f"{data_path}/clustering/full_cd_clus_21_40_49.csv"

# output_path = f"{data_path}/frequencies/"
comms = pd.read_csv(file_path2)
# comms = pd.read_csv(file_path2)[['agglomerateto_39', 'source_ID']]

dists = pd.read_csv(file_path)
dists['source_cluster'] = dists['source_cluster'].replace({'T other':'T_other','Non-Cl Mo':'Non-Cl_Mo','Int Mo':'Int_Mo','Cl Mo':'Cl_Mo','Cl MAC':'Cl_Mac','Alt MAC':'Alt_Mac', 'NK cell':'NK','DCs cell':'DC','Neutrophils':'Neutrophil','B cell': 'B', 'Endothelial cell': 'Endothelial', 'Mast cell':'Mast'})
dists['target_cluster'] = dists['target_cluster'].replace({'T other':'T_other','Non-Cl Mo':'Non-Cl_Mo','Int Mo':'Int_Mo','Cl Mo':'Cl_Mo','Cl MAC':'Cl_Mac','Alt MAC':'Alt_Mac', 'NK cell':'NK','DCs cell':'DC','Neutrophils':'Neutrophil','B cell': 'B', 'Endothelial cell': 'Endothelial', 'Mast cell':'Mast'})


In [4]:
dists

,source_ID,target_ID,distance,source_X,source_Y,target_X,target_Y,source_cluster,target_cluster,Patient_ID
0,D223_1133,D223_1,12.041595,183,156,174,164,Cancer,Cancer,LUAD_D223
1,D223_2,D223_1,24.166092,164,186,174,164,Cancer,Cancer,LUAD_D223
2,D223_2307,D223_1,13.892444,181,176,174,164,Cancer,Cancer,LUAD_D223
3,D223_3639,D223_1,19.104973,193,162,174,164,Cancer,Cancer,LUAD_D223
4,D223_4,D223_1,19.026298,175,183,174,164,Cl_Mac,Cancer,LUAD_D223
...,...,...,...,...,...,...,...,...,...,...
17104169,D165_3084,D165_999,16.124515,912,401,904,415,Cancer,Cancer,LUAD_D165
17104170,D165_3086,D165_999,20.248457,915,432,904,415,Cancer,Cancer,LUAD_D165
17104171,D165_909,D165_999,23.021729,905,392,904,415,Cancer,Cancer,LUAD_D165
17104172,D165_926,D165_999,20.615528,899,395,904,415,Cancer,Cancer,LUAD_D165


In [5]:
dists_merged.columns

NameError: name 'dists_merged' is not defined

In [7]:
dists_merged = pd.merge(dists, comms[['source_ID', 'agglomerateto_40']], on='source_ID', how='inner').rename(columns={'agglomerateto_40':'community'})

dists_merged = dists_merged[['source_ID', 'target_ID','source_cluster', 'target_cluster', 'Patient_ID', 'community']]
# comms
dists_merged

,source_ID,target_ID,source_cluster,target_cluster,Patient_ID,community
0,D223_1133,D223_1,Cancer,Cancer,LUAD_D223,19
1,D223_2,D223_1,Cancer,Cancer,LUAD_D223,16
2,D223_2307,D223_1,Cancer,Cancer,LUAD_D223,6
3,D223_3639,D223_1,Cancer,Cancer,LUAD_D223,4
4,D223_4,D223_1,Cl_Mac,Cancer,LUAD_D223,4
...,...,...,...,...,...,...
17104169,D165_3084,D165_999,Cancer,Cancer,LUAD_D165,17
17104170,D165_3086,D165_999,Cancer,Cancer,LUAD_D165,17
17104171,D165_909,D165_999,Cancer,Cancer,LUAD_D165,8
17104172,D165_926,D165_999,Cancer,Cancer,LUAD_D165,39


In [8]:
# set number of permutations
num_perms = 300
# Significance threshold
p_threshold = 0.01
np.random.seed(42)

# calculate mean number of celltype B in surrounding of celltype A's per treatment, ROI
def aggregate_histo(data):
    counts = data.groupby(by=['Patient_ID','community','source_ID', 'source_cluster', 'target_cluster']).size().reset_index(name='mean')
    means = counts.groupby(by=['Patient_ID','community','source_cluster', 'target_cluster'])['mean'].mean().reset_index()
    return means

# Shuffle the cell types B in a ROI to randomize the neighbours of cell type A
def shuffle_labels(data):
    shuffled_data = data.copy()
    new_dfs = []
    for treatment in shuffled_data['treatment'].unique():
        for roi in shuffled_data.loc[shuffled_data['treatment'] == treatment, 'ROI_ID'].unique():
            df = shuffled_data[(shuffled_data['treatment'] == treatment) & (shuffled_data['ROI_ID'] == roi)].copy()
            df['target_cluster'] = np.random.permutation(df['target_cluster'])
            new_dfs.append(df)
    new_df = pd.concat(new_dfs)
    return new_df


# Shuffle the cell types B in a community 
def shuffle_labels_across_comms(data):
    shuffled_data = data.copy()
    new_dfs = []
    for community in shuffled_data['community'].unique():
        df = shuffled_data[shuffled_data['community'] == community].copy()
        df['target_cluster'] = np.random.permutation(df['target_cluster'])
        new_dfs.append(df)
    new_df = pd.concat(new_dfs)
    return new_df



# Shuffle the data then obtain mean of number of neighbours per celltype
# Only use for permutation
def shuffle_and_aggregate(data):
    data_shuffled = shuffle_labels_across_comms(data)
    return aggregate_histo(data_shuffled)


def calc_p_vals(dat_baseline, dat_perm, n_perm, p_thresh=0.01):
    # Merge bas with permutation
    dat_perm = pd.merge(dat_perm, dat_baseline[['Patient_ID','community','source_cluster', 'target_cluster', 'mean']],
                        on=['Patient_ID','community','source_cluster', 'target_cluster'], suffixes=('_perm', '_obs'), how='outer')

    # Replace NA values with 0
    dat_perm['mean_perm'] = dat_perm['mean_perm'].fillna(0)
    dat_perm['mean_obs'] = dat_perm['mean_obs'].fillna(0)
    
    # only for printing
    dat_perm_res = dat_perm.groupby(by=['Patient_ID','community','source_cluster', 'target_cluster', 'mean_obs'])['mean_perm'].mean().reset_index()

    # Calculating p_gt and p_lt
    test_df = dat_perm.groupby(['Patient_ID','community', 'source_cluster', 'target_cluster'])

    dat_stat = dat_perm.groupby(['Patient_ID','community','source_cluster', 'target_cluster']).apply(
        lambda x: pd.Series({
            # p_gt p value is low when observed is high
            'p_gt': 1 if x['mean_obs'].max() == 0 else (sum(x['mean_perm'] >= x['mean_obs']) + 1) / (n_perm + 1),
            'p_lt': (n_perm - sum(x['mean_perm'] > x['mean_obs']) + 1) / (n_perm + 1)
        })).reset_index()

    # Determining direction and p-values
    dat_stat['direction'] = dat_stat['p_gt'] < dat_stat['p_lt']
    dat_stat['p'] = np.where(dat_stat['direction'], dat_stat['p_gt'], dat_stat['p_lt'])
    
    # Determining significance
    dat_stat['sig'] = dat_stat['p'] < p_thresh
    dat_stat['sigval'] = dat_stat['sig'].astype(int) * np.sign(dat_stat['direction'] - 0.5)
    
    return dat_stat, dat_perm_res, dat_perm, test_df


perms = []
for i in range(num_perms):
    perms.append(shuffle_and_aggregate(dists_merged[['Patient_ID', 'source_ID', 'source_cluster', 'target_cluster', 'community']]))

perm = pd.concat(perms, ignore_index=True)
bas = aggregate_histo(dists_merged[['Patient_ID','community','source_ID', 'source_cluster', 'target_cluster']])
# shuf = shuffle_and_aggregate(dists_merged)
dat_perm_pvals, dat_perm_res_meaned, dat_perm_original_counts, test_df = calc_p_vals(bas, perm, num_perms, p_thresh=p_threshold)



C:\Users\jhroe\AppData\Local\Temp\ipykernel_16260\3898779141.py:61: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dat_stat = dat_perm.groupby(['Patient_ID','community','source_cluster', 'target_cluster']).apply(


In [12]:
# Save the dataframes for Log FC calculation
dat_perm_pvals.to_csv(f"{data_path}permutation_results_300_pvals_280624.csv")
dat_perm_res_meaned.to_csv(f"{data_path}permutation_results_300_meaned_280624.csv")
dat_perm_original_counts.to_csv(f"{data_path}permutation__300_original_values_280624.csv")

In [13]:
dat_perm_pvals

,Patient_ID,community,source_cluster,target_cluster,p_gt,p_lt,direction,p,sig,sigval
0,LUAD_D001,1,Cancer,Alt_Mac,1.000000,0.946844,False,0.946844,False,-0.0
1,LUAD_D001,1,Cancer,B,1.000000,0.910299,False,0.910299,False,-0.0
2,LUAD_D001,1,Cancer,Cancer,0.039867,0.966777,True,0.039867,False,0.0
3,LUAD_D001,1,Cancer,Cl_Mac,1.000000,0.003322,False,0.003322,True,-1.0
4,LUAD_D001,1,Cancer,Cl_Mo,1.000000,0.933555,False,0.933555,False,-0.0
...,...,...,...,...,...,...,...,...,...,...
1309496,LUAD_D416,40,Treg,T_other,1.000000,0.983389,False,0.983389,False,-0.0
1309497,LUAD_D416,40,Treg,Tc,1.000000,0.986711,False,0.986711,False,-0.0
1309498,LUAD_D416,40,Treg,Th,1.000000,0.980066,False,0.980066,False,-0.0
1309499,LUAD_D416,40,Treg,Treg,1.000000,0.953488,False,0.953488,False,-0.0


In [42]:
dat_perm_pvals[dat_perm_pvals['Patient_ID'] == 'LUAD_D001']

,Patient_ID,community,source_cluster,target_cluster,p_gt,p_lt,direction,p,sig,sigval
0,LUAD_D001,1,Cancer,Alt_Mac,1.000000,0.967213,False,0.967213,False,-0.0
1,LUAD_D001,1,Cancer,B,1.000000,0.934426,False,0.934426,False,-0.0
2,LUAD_D001,1,Cancer,Cancer,0.098361,0.934426,True,0.098361,False,0.0
3,LUAD_D001,1,Cancer,Cl_Mac,1.000000,0.016393,False,0.016393,False,-0.0
4,LUAD_D001,1,Cancer,Cl_Mo,1.000000,0.950820,False,0.950820,False,-0.0
...,...,...,...,...,...,...,...,...,...,...
3307,LUAD_D001,40,Unknown,Neutrophil,1.000000,0.950820,False,0.950820,False,-0.0
3308,LUAD_D001,40,Unknown,T_other,1.000000,0.983607,False,0.983607,False,-0.0
3309,LUAD_D001,40,Unknown,Tc,1.000000,0.983607,False,0.983607,False,-0.0
3310,LUAD_D001,40,Unknown,Treg,1.000000,0.934426,False,0.934426,False,-0.0


In [17]:
dat_perm_pvals[(dat_perm_pvals['sigval'] != 0)] 

,Patient_ID,community,source_cluster,target_cluster,p_gt,p_lt,direction,p,sig,sigval
3,LUAD_D001,1,Cancer,Cl_Mac,1.000000,0.003322,False,0.003322,True,-1.0
115,LUAD_D001,3,Alt_Mac,Tc,0.003322,1.000000,True,0.003322,True,1.0
127,LUAD_D001,3,Cancer,Mast,1.000000,0.003322,False,0.003322,True,-1.0
135,LUAD_D001,3,Cancer,Unknown,0.003322,1.000000,True,0.003322,True,1.0
147,LUAD_D001,3,Cl_Mac,Non-Cl_Mo,0.003322,1.000000,True,0.003322,True,1.0
...,...,...,...,...,...,...,...,...,...,...
1309278,LUAD_D416,39,Neutrophil,Non-Cl_Mo,0.006645,1.000000,True,0.006645,True,1.0
1309335,LUAD_D416,39,Th,Cancer,1.000000,0.003322,False,0.003322,True,-1.0
1309336,LUAD_D416,39,Th,Cl_Mac,0.006645,0.996678,True,0.006645,True,1.0
1309369,LUAD_D416,39,Unknown,Cancer,1.000000,0.003322,False,0.003322,True,-1.0


In [27]:
dat_perm_pvals[(dat_perm_pvals['sigval'] != 0) & (dat_perm_pvals['community'].isin([26]))& (dat_perm_pvals['target_cluster'] == 'Treg')&(dat_perm_pvals['source_cluster'] == 'B') ]['sigval']

sigval
-1.0    49
 1.0    34
Name: count, dtype: int64